Para realizar nuestro modelo de recomendación, que es nuestra meta final, comenzaré realizando las transformaciones correspondientes que la empresa solicita para tener los datos más limpios posibles. Sin embargo, continuaré realizando más limpieza posteriormente, ya que, después de todo, soy el Data Scientist 😆.

Comenzaremos importando algunas librerías que se utilizarán para realizar el ETL y luego cargamos los datasets que nos han entregado.

In [1]:
import pandas as pd
import numpy as np
import ast
import json

In [2]:
df = pd.read_csv("movies_dataset.csv", sep=",")
credit = pd.read_csv("credits.csv", sep=",")

C:\Users\Juli\AppData\Local\Temp\ipykernel_16532\2986500485.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_dataset.csv", sep=",")


Analizamos un poco que tenemos en el primer archivo. 🔎

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Procederemos a eliminar las columnas indicadas por la empresa. Esta acción nos beneficiará al reducir el número de columnas y la cantidad de datos en el dataset.

In [4]:
df = df.drop(columns=["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"])

Por otro lado, se observa que hay columnas a las cuales tendremos que desanidar ya que contienen diccionarios y/o listas de diccionarios en sus filas.
Se comienza trabajando con la columna belongs_to_collection, luego continuamos con genres, production_companies, production_countries y spoken_languages.

In [5]:
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return np.nan

# Se convierte la columna 'belongs_to_collection' de string a diccionario
df['belongs_to_collection'] = df['belongs_to_collection'].apply(safe_literal_eval)

# Se desanidan los datos del diccionario en columnas separadas
df_belongs_to_collection = pd.json_normalize(df['belongs_to_collection'])

df_belongs_to_collection.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4491 non-null   float64
 1   name           4491 non-null   object 
 2   poster_path    3948 non-null   object 
 3   backdrop_path  3263 non-null   object 
dtypes: float64(1), object(3)
memory usage: 1.4+ MB


In [6]:
# Se cambian los nombres de las columnas desanidadas ya que sino se pueden confundir con las columnas ya existentes
df_belongs_to_collection.rename(columns={'id': 'idcollection', 'name': 'name_collection'}, inplace=True)

# Se eliminan dos de las cuatro columnas ya que no se utilizarán
df_belongs_to_collection.drop(['poster_path', 'backdrop_path'], axis=1, inplace=True)

In [7]:
# Se unen las dos columnas al resto del dataset
df = pd.concat([df, df_belongs_to_collection], axis=1)

In [8]:
#Eliminamos la columna belongs_to_collection
df.drop(['belongs_to_collection'], axis=1, inplace=True)

Continuamos con las otras columnas mensionadas anteorimente, pero en estos casos desanidamos y tomamos solo la información que nos interesa para la creación de las nuevas columas. Los códigos que se utilizan para realizar la desanidación son diferentes al anterior ya que contienen listas de diccionarios.

In [9]:
# Se realiza una función para desanidar la columna genres
def desanidar_genres(row):
    try:
        genres_list = eval(row)  # Se convierte la cadena en una lista de diccionarios
        genres_names = [genre['name'] for genre in genres_list]  # Se extraen solo los nombres de los géneros
        id_genres = [genre['id'] for genre in genres_list]  # Se extraen los IDs de los géneros
        return genres_names, id_genres
    except:
        return None, None

# Se aplica la función a la columna 'genres' y se crean dos nuevas columnas 'genres_names' e 'idgenres'
df['genres_names'], df['idgenres'] = zip(*df['genres'].map(desanidar_genres))

In [10]:
# Se elimina la columna principal genres 
df.drop(['genres'], axis=1, inplace=True)

In [11]:
# Se realiza una función para desanidar la columna 'production_companies'
def desanidar_companies(row):
    try:
        companies_list = eval(row)  # Se convierte la cadena en una lista de diccionarios
        companies_names = [companie['name'] for companie in companies_list]  # Se extraen solo los nombres 
        id_companies = [companie['id'] for companie in companies_list]  # Se extraen los IDs 
        return companies_names, id_companies
    except:
        return None, None

# Se aplica la función a la columna 'production_companies' y se crean dos nuevas columnas 'companies_names' e 'idcompanies'
df['companies_names'], df['idcompanies'] = zip(*df['production_companies'].map(desanidar_companies))

In [12]:
# Se realiza una función para desanidar la columna 'production_countries'
def desanidar_countries(row):
    try:
        countriescompanies_list = eval(row)  # Se convierte la cadena en una lista de diccionarios
        countries_names = [countries['name'] for countries in countriescompanies_list]  # Se extraen solo los nombres
        id_countries = [countries['iso_3166_1'] for countries in countriescompanies_list]  # Se extraen los IDs 
        return countries_names, id_countries
    except:
        return None, None

# Se aplica la función a la columna 'production_countries' y se crean dos nuevas columnas 'countries_names' e 'idcountries'
df['countries_names'], df['idcountries'] = zip(*df['production_countries'].map(desanidar_countries))

In [13]:
# Se realiza una función para desanidar la columna 'spoken_languages'
def desanidar_languages(row):
    try:
        lenguages_list = eval(row)  # Convertir la cadena en una lista de diccionarios
        lenguages_names = [lenguages['name'] for lenguages in lenguages_list]  # Se extraen solo los nombres
        id_lenguages = [lenguages['iso_639_1'] for lenguages in lenguages_list]  # Se extraen los IDs 
        return lenguages_names, id_lenguages
    except:
        return None, None

# Se aplica la función a la columna 'spoken_languages' y se crean dos nuevas columnas 'lenguages_names' e 'id_lenguages'
df['lenguages_names'], df['id_lenguages'] = zip(*df['spoken_languages'].map(desanidar_languages))

In [14]:
# Se eliminan las columnas que ya se desanidaron 
df.drop(['production_companies'], axis=1, inplace=True)
df.drop(['production_countries'], axis=1, inplace=True)
df.drop(['spoken_languages'], axis=1, inplace=True)
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,idcollection,name_collection,genres_names,idgenres,companies_names,idcompanies,countries_names,idcountries,lenguages_names,id_lenguages
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,"[Animation, Comedy, Family]","[16, 35, 10751]",[Pixar Animation Studios],[3],[United States of America],[US],[English],[en]
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,NaN,NaN,"[Adventure, Fantasy, Family]","[12, 14, 10751]","[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",[United States of America],[US],"[English, Français]","[en, fr]"
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,119050.0,Grumpy Old Men Collection,"[Romance, Comedy]","[10749, 35]","[Warner Bros., Lancaster Gate]","[6194, 19464]",[United States of America],[US],[English],[en]
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,NaN,NaN,"[Comedy, Drama, Romance]","[35, 18, 10749]",[Twentieth Century Fox Film Corporation],[306],[United States of America],[US],[English],[en]
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,96871.0,Father of the Bride Collection,[Comedy],[35],"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",[United States of America],[US],[English],[en]


Analizamos el archivo credits.

In [15]:
credit.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Observamos que tiene el mismo problema de columnas anidadas así que realizamos los mismos pasos utilizados anteriormente.

In [16]:
# Se realiza una función para desanidar la columna 'cast'
def desanidar_cast(row):
    try:
        cast_list = eval(row)  # Se convierte la cadena en una lista de diccionarios
        cast_names = [cast['name'] for cast in cast_list]  # Extraer solo los nombres 
        id_cast = [cast['cast_id'] for cast in cast_list]  # Extraer los IDs 
        return cast_names, id_cast
    except:
        return None, None

# Se aplica la función a la columna 'cast' y se crean dos nuevas columnas 'cast_names' e 'id_cast'
credit['names_actors'], credit['id_cast'] = zip(*credit['cast'].map(desanidar_cast))

Respecto a la columa crew, solo nos interesa conocer al director así que tomaremos solo la que corresponda.

In [17]:
# Se convierte la columna 'crew' de una cadena a una lista de diccionarios
credit['crew'] = credit['crew'].apply(ast.literal_eval)

# Se extrae el 'director' de la columna 'crew'
credit['director'] = credit['crew'].apply(lambda x: next((d['name'] for d in x if d['job'] == 'Director'), None))

Observamos que se hayan realizado los cambios correctamente.

In [18]:
credit.head()

,cast,crew,id,names_actors,id_cast,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[14, 15, 16, 17, 18, 19, 20, 26, 22, 23, 24, 2...",John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[1, 8, 2, 24, 10, 25, 26, 11, 14, 13, 31, 12, ...",Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[2, 3, 4, 5, 6, 9, 10]",Howard Deutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[1, 2, 3, 4, 5, 6, 8, 10, 20, 21]",Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[1, 2, 3, 4, 13, 14, 15, 16, 17, 18, 19, 20]",Charles Shyer


Eliminaremos las columnas originales ya desanidadas y además, se eliminará la columna id_cast ya que luego de ver el resultado final, me di cuenta que no se utilizará ❌ y sería en vano tenerla.

In [19]:
credit.drop(['crew', 'cast', 'id_cast'], axis=1, inplace=True)
credit.head()

,id,names_actors,director
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer


Una vez completado ese trabajo, nos disponemos a unir todo en un solo dataset. Antes de hacer la unión correspondiente hay que analizar que todo este como corresponde para que no nos arroje ningún error y también para buscar 🔎 por medio de que columna se hará la combinación.

In [20]:
df.info()
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45466 non-null  object 
 1   id                 45466 non-null  object 
 2   original_language  45455 non-null  object 
 3   overview           44512 non-null  object 
 4   popularity         45461 non-null  object 
 5   release_date       45379 non-null  object 
 6   revenue            45460 non-null  float64
 7   runtime            45203 non-null  float64
 8   status             45379 non-null  object 
 9   tagline            20412 non-null  object 
 10  title              45460 non-null  object 
 11  vote_average       45460 non-null  float64
 12  vote_count         45460 non-null  float64
 13  idcollection       4491 non-null   float64
 14  name_collection    4491 non-null   object 
 15  genres_names       45466 non-null  object 
 16  idgenres           454

La unión se realizará a través de la columna id pero notamos que el tipo de dato no corresponde así que se procede a cambiar ==> type = int 

In [21]:
df['id'] = pd.to_numeric(df['id'], errors='coerce', downcast='integer')

Ahora que todo esta como corresponde ✔ procedemos a unirlos.

In [22]:
df = pd.merge(df, credit, on='id')

Y por último observamos que todo haya salido bien 🧐

In [23]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,genres_names,idgenres,companies_names,idcompanies,countries_names,idcountries,lenguages_names,id_lenguages,names_actors,director
0,30000000,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,"[Animation, Comedy, Family]","[16, 35, 10751]",[Pixar Animation Studios],[3],[United States of America],[US],[English],[en],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,65000000,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[Adventure, Fantasy, Family]","[12, 14, 10751]","[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",[United States of America],[US],"[English, Français]","[en, fr]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston
2,0,15602.0,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[Romance, Comedy]","[10749, 35]","[Warner Bros., Lancaster Gate]","[6194, 19464]",[United States of America],[US],[English],[en],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch
3,16000000,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,"[Comedy, Drama, Romance]","[35, 18, 10749]",[Twentieth Century Fox Film Corporation],[306],[United States of America],[US],[English],[en],"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker
4,0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,[Comedy],[35],"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",[United States of America],[US],[English],[en],"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer


La empresa también nos comentó que quería que haya una columna donde se pueda consultar el retorno de la inversión de la película 💸. 
Para esto analizamos algunas de las columnas ya existentes...

In [24]:
# Observamos si los campos revenue y budget contienen valores nulos

print(df[['revenue', 'budget']].isnull().sum())

revenue    3
budget     0
dtype: int64


In [25]:
# Solo una de las columnas contiene valores nulos así que los reemplazamos por 0

df["revenue"].fillna(0, inplace = True)

In [26]:
# Se deberá cambiar el tipo de dato a la columna budget

df['budget'] = df['budget'].astype(float)

In [27]:
# Creamos la columna que nos solicitó el cliente dividiendo las columnas correspondientes (revenue ➗ budget)

def retorno(column):
    if column["budget"] == 0:
        return 0
    else:
        return column["revenue"] / column["budget"]

df["return"] = df.apply(retorno, axis=1)

print(df["return"].head())

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64


Por último, se creará una nueva columna solamente con el año del estreno de la película pero antes analicémoslo un poquito 😉.

In [28]:
# Observamos si el campo release date contiene valores nulos

print(df['release_date'].isnull().sum())

87


In [29]:
# Se eliminan valores nulos del campo release date

df = df[df['release_date'].notnull()]

Las fechas deben tener un mismo formato y aunque observando algunas filas vemos que están correctas, no debemos dejar pasar ninguna así que aplicamos la fórmula para que, en el caso de que alguna no esté correctamente, se corrija.

In [30]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

Ahora si, creemos la nueva columna.

In [31]:
df['release_year'] = df['release_date'].dt.year

print(df[['release_date', 'release_year']])

      release_date  release_year
0       1995-10-30          1995
1       1995-12-15          1995
2       1995-12-22          1995
3       1995-12-22          1995
4       1995-02-10          1995
...            ...           ...
45532   1991-05-13          1991
45534   2011-11-17          2011
45535   2003-08-01          2003
45536   1917-10-21          1917
45537   2017-06-09          2017

[45451 rows x 2 columns]


Y este es el resultado ❗ pero no el resultado final 😯

In [32]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,companies_names,idcompanies,countries_names,idcountries,lenguages_names,id_lenguages,names_actors,director,return,release_year
0,30000000.0,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,[Pixar Animation Studios],[3],[United States of America],[US],[English],[en],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,12.451801,1995
1,65000000.0,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",[United States of America],[US],"[English, Français]","[en, fr]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,4.043035,1995
2,0.0,15602.0,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[Warner Bros., Lancaster Gate]","[6194, 19464]",[United States of America],[US],[English],[en],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,0.000000,1995
3,16000000.0,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,[Twentieth Century Fox Film Corporation],[306],[United States of America],[US],[English],[en],"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,5.090760,1995
4,0.0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",[United States of America],[US],[English],[en],"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,0.000000,1995


Guardemos el nuevo archivo. 💾

In [33]:
df.to_csv('Movies_ETL.csv',index=False)